# MO-IT143: Ethical Hacking - Python Snippets

**Mapúa Malayan Digital College**

**Course:** MO-IT143 Ethical Hacking

**Welcome!** This Jupyter Notebook contains basic Python script snippets relevant to the topics and Capture the Flag (CTF) challenges covered in your Ethical Hacking course. These examples are designed to help you practice the Python skills needed for security assessments and CTFs.

**Ethical Disclaimer:** Remember the core principles of ethical hacking. **DO NOT** run scanning, exploitation, or analysis scripts against any system or network without explicit, written permission. Unauthorized access is illegal and unethical. Use this knowledge responsibly for learning within the course's controlled environments (like CTFd) and for defensive purposes.

## Prerequisites

1.  **Python 3:** Ensure you have Python 3 installed.
2.  **Jupyter Notebook:** You need Jupyter Notebook or JupyterLab.
3.  **Libraries:** Some snippets require external libraries. Install them using pip:
    ```bash
    pip install requests
    # Other libraries like 'cryptography' might be needed for specific tasks later
    # pip install cryptography 
    ```

## Foundational Snippets (Relevant to Multiple Weeks)

These are common tasks you'll encounter.

### 1. DNS Lookup (Finding IP Address)

**Concept:** Resolve a domain name to its IP address. Fundamental for targeting.
*(Relevant Background for Reconnaissance)*

In [ ]:
import socket
import sys

def get_ip_address(domain_name):
    """Resolves a domain name to its IPv4 address."""
    try:
        ip_address = socket.gethostbyname(domain_name)
        print(f"[*] The IP address for {domain_name} is: {ip_address}")
        return ip_address
    except socket.gaierror as e: 
        print(f"[!] Could not resolve hostname {domain_name}: {e}")
        return None
    except Exception as e:
        print(f"[!] An unexpected error occurred: {e}")
        return None

# --- Example Usage ---
# Use a domain you have permission to query, or a public one like example.com
target_domain = "example.com" 
ip = get_ip_address(target_domain)
if ip:
    print(f"Stored IP: {ip}")

### 2. Basic Port Scanner

**Concept:** Check for open TCP ports on a target IP. Indicates running services.
*(Relevant Background for Reconnaissance)*

In [ ]:
import socket
import sys
from datetime import datetime

def basic_port_scan(target_ip, ports_to_scan):
    """Scans a list of ports on a target IP."""
    print("-" * 50)
    print(f"Scanning Target: {target_ip}")
    print(f"Ports: {ports_to_scan}")
    print(f"Time Started: {datetime.now()}")
    print("-" * 50)

    open_ports = []
    try:
        for port in ports_to_scan:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            socket.setdefaulttimeout(0.5) 
            result = sock.connect_ex((target_ip, port))
            if result == 0:
                print(f"[*] Port {port} is open")
                open_ports.append(port)
            sock.close()
    except socket.error as e:
        print(f"[!] Socket error: {e}")
    except KeyboardInterrupt:
        print("\n[!] Exiting Program.")
        sys.exit()
    except Exception as e:
        print(f"[!] An unexpected error occurred: {e}")

    print("-" * 50)
    print(f"Scan Complete. Open ports: {open_ports}")
    print("-" * 50)
    return open_ports

# --- Example Usage ---
# WARNING: Only scan hosts you have permission for (e.g., CTFd targets, scanme.nmap.org)
target_host_for_scan = "scanme.nmap.org" # Explicitly for testing scanners
try:
    target_ip_addr = socket.gethostbyname(target_host_for_scan)
    common_ports = [21, 22, 23, 80, 443] # Example ports
    basic_port_scan(target_ip_addr, common_ports)
except socket.gaierror:
     print(f"Could not resolve {target_host_for_scan}")

## Week 1: Flagged for Extraction CTF & Python Fundamentals

**Task:** Retrieve a hidden flag from a text file using Python.

In [ ]:
import os

# Assume 'mystery_text_file.txt' is in the same directory as the notebook
# Create a dummy file for demonstration:
dummy_filename = 'mystery_text_file.txt'
dummy_content = """Some introductory text.
Lots of lines here.
Maybe the flag is hidden somewhere?
Look carefully for CTF{Th1s_Is_A_S4mpl3_Fl4g}
More text follows.
End of file.
"""
try:
    with open(dummy_filename, 'w') as f:
        f.write(dummy_content)
    print(f"[*] Created dummy file: {dummy_filename}")
except IOError as e:
    print(f"[!] Error creating dummy file: {e}")

def find_flag_in_file(filename, flag_format="CTF{"):
    """Reads a file line by line and searches for a specific flag format."""
    print(f"\n[*] Searching for flag in {filename}...")
    try:
        with open(filename, 'r') as f:
            for line_num, line in enumerate(f, 1):
                # Basic string searching
                if flag_format in line:
                    # Try to extract the full flag (assuming it ends with '}')
                    start_index = line.find(flag_format)
                    end_index = line.find('}', start_index) 
                    if end_index != -1:
                        flag = line[start_index:end_index+1].strip()
                        print(f"[*] Potential Flag Found on line {line_num}: {flag}")
                        return flag
                    else:
                        # If no closing brace, print the line anyway
                        print(f"[*] Potential Flag Format Found on line {line_num} (incomplete?): {line.strip()}")
                        
        print(f"[!] Flag format '{flag_format}' not found in {filename}.")
        return None
        
    except FileNotFoundError:
        print(f"[!] Error: File '{filename}' not found.")
        return None
    except Exception as e:
        print(f"[!] An error occurred: {e}")
        return None

# --- Example Usage ---
# Make sure the 'mystery_text_file.txt' (or your actual CTF file) exists
found_flag = find_flag_in_file(dummy_filename)

# Clean up the dummy file
# if os.path.exists(dummy_filename):
#    os.remove(dummy_filename)
#    print(f"[*] Removed dummy file: {dummy_filename}")

## Week 2: Data Heist CTF & File Operations

**Task:** Extract encoded messages from a JSON file.

In [ ]:
import json
import base64 # Often used for simple encoding in CTFs

# Assume 'security_logs.json' exists
# Create a dummy JSON file for demonstration:
dummy_json_filename = 'security_logs.json'
dummy_json_data = {
    "log_id": "log123",
    "timestamp": "2025-04-23T10:00:00Z",
    "events": [
        {"type": "login", "user": "admin", "status": "success"},
        {"type": "data_access", "user": "guest", "details": "Normal access"},
        {"type": "encoded_message", "payload": "SFRCe0pTT05fSEVJU1RfRkxBR30="} # Base64 for 'CTF{JSON_HEIST_FLAG}'
    ],
    "metadata": {"source": "server01"}
}
try:
    with open(dummy_json_filename, 'w') as f:
        json.dump(dummy_json_data, f, indent=4)
    print(f"[*] Created dummy file: {dummy_json_filename}")
except IOError as e:
    print(f"[!] Error creating dummy file: {e}")

def extract_encoded_payloads(json_filename, key_to_find='payload'):
    """Loads a JSON file and extracts values associated with a specific key."""
    print(f"\n[*] Processing JSON file: {json_filename}")
    extracted_data = []
    try:
        with open(json_filename, 'r') as f:
            data = json.load(f)
            
            # This needs modification based on the *actual* JSON structure
            # Example: If payloads are in a list of events
            if 'events' in data and isinstance(data['events'], list):
                for event in data['events']:
                    if isinstance(event, dict) and key_to_find in event:
                        payload = event[key_to_find]
                        print(f"[*] Found encoded payload: {payload}")
                        extracted_data.append(payload)
                        # Try to decode it (assuming Base64)
                        try:
                            decoded_bytes = base64.b64decode(payload)
                            decoded_string = decoded_bytes.decode('utf-8')
                            print(f"    Decoded (UTF-8): {decoded_string}")
                            # Check if it looks like a flag
                            if decoded_string.startswith("CTF{") and decoded_string.endswith("}"):
                                print("    [!] Looks like a flag!")
                        except (base64.binascii.Error, UnicodeDecodeError) as decode_err:
                            print(f"    Could not decode as Base64/UTF-8: {decode_err}")
                        except Exception as e:
                             print(f"    An error occurred during decoding: {e}")

            else:
                 print("[!] Could not find 'events' list in JSON structure. Adapt the script!")

        return extracted_data

    except FileNotFoundError:
        print(f"[!] Error: File '{json_filename}' not found.")
        return []
    except json.JSONDecodeError as e:
        print(f"[!] Error decoding JSON from '{json_filename}': {e}")
        return []
    except Exception as e:
        print(f"[!] An error occurred: {e}")
        return []

# --- Example Usage ---
payloads = extract_encoded_payloads(dummy_json_filename)

# Clean up the dummy file
# import os
# if os.path.exists(dummy_json_filename):
#     os.remove(dummy_json_filename)
#     print(f"[*] Removed dummy file: {dummy_json_filename}")

## Week 3: Hacker's Diary CTF & Log Parsing

**Task:** Analyze system logs (text file) and extract flagged entries.

In [ ]:
# Assume 'access_logs.txt' exists
# Create a dummy log file for demonstration:
dummy_log_filename = 'access_logs.txt'
dummy_log_content = """[2025-04-23 11:00:01] INFO: User 'alice' logged in from 192.168.1.10
[2025-04-23 11:05:23] WARN: Failed login attempt for user 'root' from 10.0.0.5
[2025-04-23 11:05:25] WARN: Failed login attempt for user 'root' from 10.0.0.5
[2025-04-23 11:07:00] INFO: File '/data/report.pdf' accessed by 'bob'
[2025-04-23 11:08:15] FLAG: Suspicious activity detected. User 'eve' attempted unauthorized access. CTF{L0G_F1L3_M4ST3R}
[2025-04-23 11:10:00] INFO: Service 'webserver' restarted.
"""
try:
    with open(dummy_log_filename, 'w') as f:
        f.write(dummy_log_content)
    print(f"[*] Created dummy file: {dummy_log_filename}")
except IOError as e:
    print(f"[!] Error creating dummy file: {e}")

def parse_log_for_keywords(log_filename, keywords):
    """Reads a log file and prints lines containing specific keywords."""
    print(f"\n[*] Parsing log file: {log_filename} for keywords: {keywords}")
    found_lines = []
    try:
        with open(log_filename, 'r') as f:
            for line_num, line in enumerate(f, 1):
                # Check if *any* of the keywords are in the line (case-insensitive)
                if any(keyword.lower() in line.lower() for keyword in keywords):
                    print(f"[*] Match on line {line_num}: {line.strip()}")
                    found_lines.append(line.strip())
                    # If 'FLAG' or 'CTF{' is a keyword, try extracting the flag
                    if "flag" in keywords or "ctf{" in keywords:
                         if "CTF{" in line:
                            start_index = line.find("CTF{")
                            end_index = line.find('}', start_index)
                            if end_index != -1:
                                flag = line[start_index:end_index+1].strip()
                                print(f"    [!] Potential Flag Extracted: {flag}")

        if not found_lines:
            print(f"[!] No lines found containing keywords: {keywords}")
        return found_lines
            
    except FileNotFoundError:
        print(f"[!] Error: File '{log_filename}' not found.")
        return []
    except Exception as e:
        print(f"[!] An error occurred: {e}")
        return []

# --- Example Usage ---
# Keywords to look for (can be adapted based on CTF challenge)
search_keywords = ['WARN', 'Failed', 'FLAG', 'Suspicious', 'CTF{'] 
parse_log_for_keywords(dummy_log_filename, search_keywords)

# Clean up the dummy file
# import os
# if os.path.exists(dummy_log_filename):
#     os.remove(dummy_log_filename)
#     print(f"[*] Removed dummy file: {dummy_log_filename}")

## Week 4: Hidden Web Crawler CTF & Web Security Basics

**Tasks:** Make web requests, understand basic SQL injection concepts.

### 4a. Making HTTP Requests (GET/POST)

**Concept:** Interact with web servers using Python's `requests` library.

In [ ]:
import requests

def make_get_request(url):
    """Makes a simple GET request to a URL."""
    print(f"\n[*] Making GET request to: {url}")
    try:
        headers = {'User-Agent': 'MMDC-EthicalHacking-Bot/1.0'}
        response = requests.get(url, headers=headers, timeout=10, allow_redirects=True)
        response.raise_for_status() # Raise an exception for bad status codes (4xx or 5xx)
        
        print(f"[*] Status Code: {response.status_code}")
        print(f"[*] Content Type: {response.headers.get('Content-Type', 'N/A')}")
        # Print only the first 500 chars of the content
        print("[*] Content (first 500 chars):")
        print(response.text[:500])
        return response
    except requests.exceptions.RequestException as e:
        print(f"[!] Error making GET request to {url}: {e}")
        return None
    except Exception as e:
        print(f"[!] An unexpected error occurred: {e}")
        return None

def make_post_request(url, data_payload):
    """Makes a POST request with data."""
    print(f"\n[*] Making POST request to: {url} with data: {data_payload}")
    try:
        headers = {'User-Agent': 'MMDC-EthicalHacking-Bot/1.0'}
        response = requests.post(url, headers=headers, data=data_payload, timeout=10)
        response.raise_for_status()
        
        print(f"[*] Status Code: {response.status_code}")
        print("[*] Response Content (first 500 chars):")
        print(response.text[:500])
        return response
    except requests.exceptions.RequestException as e:
        print(f"[!] Error making POST request to {url}: {e}")
        return None
    except Exception as e:
        print(f"[!] An unexpected error occurred: {e}")
        return None

# --- Example Usage ---
# Use websites you have permission to test, or public test sites
get_url = "http://httpbin.org/get" # A site for testing HTTP requests
make_get_request(get_url)

post_url = "http://httpbin.org/post"
post_data = {'username': 'testuser', 'query': 'searchterm'}
make_post_request(post_url, post_data)

### 4b. Basic SQL Injection String Construction (Illustrative)

**Concept:** Show how malicious strings are crafted. **WARNING:** Do not execute these against live systems without permission. This is purely for understanding the structure of an attack.

In [ ]:
def illustrate_sqli_strings(parameter_value):
    """Demonstrates constructing potential SQLi payloads."""
    print(f"\n[*] Original Parameter Value: '{parameter_value}'")
    
    # Very basic examples
    payload1 = f"'{parameter_value}' OR '1'='1'" # Always true condition
    payload2 = f"'{parameter_value}'; DROP TABLE users; --" # Dangerous command injection
    payload3 = f"'{parameter_value}' UNION SELECT username, password FROM users --" # Data extraction
    
    print("[*] Illustrative SQLi Payloads (DO NOT RUN ON LIVE SYSTEMS WITHOUT PERMISSION):")
    print(f"    Payload 1 (Bypass Auth): {payload1}")
    print(f"    Payload 2 (Destructive): {payload2}")
    print(f"    Payload 3 (Data Leak): {payload3}")
    
    # How it might look in a URL (simplified)
    base_url = "http://vulnerable-site.com/search?query="
    # URL encoding is usually required for special characters like ';', '--', space ('+')
    import urllib.parse
    encoded_payload1 = urllib.parse.quote_plus(payload1)
    print(f"\n[*] Example URL with Payload 1 (URL Encoded):")
    print(f"    {base_url}{encoded_payload1}")

# --- Example Usage ---
user_input = "searchterm" 
illustrate_sqli_strings(user_input)

### 4c. Checking HTTP Headers

**Concept:** Retrieve server headers for reconnaissance (Server type, technologies, etc.).

In [ ]:
import requests

def get_http_headers(url):
    """Fetches and prints HTTP headers using a HEAD request."""
    print(f"\n[*] Fetching headers for: {url}")
    try:
        headers = {'User-Agent': 'MMDC-EthicalHacking-Checker/1.0'}
        # HEAD is efficient as it doesn't download the body
        response = requests.head(url, headers=headers, timeout=10, allow_redirects=True)
        response.raise_for_status()

        print(f"[*] Status Code: {response.status_code}")
        print("[*] Headers:")
        for header, value in response.headers.items():
            print(f"    {header}: {value}")
        return response.headers
    except requests.exceptions.RequestException as e:
        print(f"[!] Error fetching headers for {url}: {e}")
        return None
    except Exception as e:
         print(f"[!] An unexpected error occurred: {e}")
         return None

# --- Example Usage ---
target_url_headers = "http://example.com"
get_http_headers(target_url_headers)

## Weeks 5-6: Milestone 1 (Cryptic Vault) & Cryptography Basics

**Tasks:** Crack weak password hash, understand hashing.

### 5a. Hashing Data (MD5, SHA-256)

**Concept:** Generate fixed-size hashes from input data. MD5/SHA1 are weak for passwords, SHA-256 is better (but needs salting in practice).

In [ ]:
import hashlib

def calculate_hashes(text):
    """Calculates MD5, SHA1, and SHA256 hashes."""
    text_bytes = text.encode('utf-8')
    
    md5_hash = hashlib.md5(text_bytes).hexdigest()
    sha1_hash = hashlib.sha1(text_bytes).hexdigest()
    sha256_hash = hashlib.sha256(text_bytes).hexdigest()
    
    print(f"\n[*] Input Text: '{text}'")
    print(f"  MD5:    {md5_hash} (Weak - Do not use for passwords)")
    print(f"  SHA1:   {sha1_hash} (Weak - Do not use for passwords)")
    print(f"  SHA256: {sha256_hash}")
    return {"md5": md5_hash, "sha1": sha1_hash, "sha256": sha256_hash}

# --- Example Usage ---
calculate_hashes("password123")
calculate_hashes("CTF{CrYpt0_Fun}")

### 5b. Basic Hash Comparison (Illustrative Cracking)

**Concept:** Compare a target hash against hashes generated from a wordlist. **WARNING:** This is slow and inefficient for real-world cracking but demonstrates the basic principle for CTFs with weak hashes.

In [ ]:
import hashlib
import time

# Assume 'wordlist.txt' and 'hashed_passwords.txt' exist
# Create dummy files:
dummy_wordlist_file = 'wordlist.txt'
dummy_words = ['password', '123456', 'qwerty', 'admin', 'secret', 'P@sswOrd!']
with open(dummy_wordlist_file, 'w') as f:
    for word in dummy_words:
        f.write(word + '\n')
print(f"[*] Created dummy file: {dummy_wordlist_file}")

dummy_hashes_file = 'hashed_passwords.txt'
# Example: MD5 hash of 'secret' is e5e9fa1ba31ecd1ae84f75caaa474f3a
# Example: SHA256 hash of 'P@sswOrd!' is 7f4e2847c4636a1aac5f6a9312307d9e9a3f8e9c1c4b7096485ba41f4d834a7f
dummy_hashes = [
    'e5e9fa1ba31ecd1ae84f75caaa474f3a', # MD5('secret')
    '7f4e2847c4636a1aac5f6a9312307d9e9a3f8e9c1c4b7096485ba41f4d834a7f' # SHA256('P@sswOrd!')
]
with open(dummy_hashes_file, 'w') as f:
    for h in dummy_hashes:
        f.write(h + '\n')
print(f"[*] Created dummy file: {dummy_hashes_file}")

def crack_hashes_from_files(hash_file, wordlist_file, hash_type='sha256'):
    """Attempts to crack hashes using a wordlist."""
    print(f"\n[*] Attempting to crack hashes in '{hash_file}' using '{wordlist_file}' (Type: {hash_type})")
    
    try:
        with open(hash_file, 'r') as hf:
            target_hashes = {line.strip() for line in hf if line.strip()}
            if not target_hashes:
                 print("[!] No hashes found in hash file.")
                 return

        print(f"[*] Loaded {len(target_hashes)} unique target hashes.")
        
        found_count = 0
        start_time = time.time()
        
        with open(wordlist_file, 'r', encoding='utf-8', errors='ignore') as wf:
            for word_num, word in enumerate(wf, 1):
                password = word.strip()
                if not password:
                    continue
                
                # Hash the word from the wordlist
                try:
                    if hash_type == 'md5':
                        current_hash = hashlib.md5(password.encode('utf-8')).hexdigest()
                    elif hash_type == 'sha1':
                         current_hash = hashlib.sha1(password.encode('utf-8')).hexdigest()
                    elif hash_type == 'sha256':
                         current_hash = hashlib.sha256(password.encode('utf-8')).hexdigest()
                    else:
                        print(f"[!] Unsupported hash type: {hash_type}")
                        return
                except Exception as hash_err:
                     # Handle potential encoding issues with wordlist entries
                     # print(f"[!] Error hashing word '{password[:20]}...': {hash_err}") 
                     continue # Skip this word
                
                # Check if the generated hash matches any target hash
                if current_hash in target_hashes:
                    print(f"[+] CRACKED! Hash: {current_hash} ==> Password: '{password}'")
                    target_hashes.remove(current_hash) # Remove found hash to avoid re-checking
                    found_count += 1
                    if not target_hashes: # Stop if all hashes are cracked
                        print("[*] All target hashes cracked!")
                        break 
                        
                # Optional: Print progress periodically
                # if word_num % 1000 == 0:
                #    print(f"[*] Processed {word_num} words...")
            
        end_time = time.time()
        print(f"\n[*] Cracking attempt finished in {end_time - start_time:.2f} seconds.")
        print(f"[*] Found {found_count} matching password(s).")
        if target_hashes:
            print(f"[!] {len(target_hashes)} hash(es) remain uncracked.")

    except FileNotFoundError as e:
        print(f"[!] Error: File not found - {e}")
    except Exception as e:
        print(f"[!] An unexpected error occurred: {e}")

# --- Example Usage ---
# Determine the hash type from the CTF or analysis
print("--- Cracking MD5 --- (Expecting 'secret')")
crack_hashes_from_files(dummy_hashes_file, dummy_wordlist_file, hash_type='md5')

print("\n--- Cracking SHA256 --- (Expecting 'P@sswOrd!')")
crack_hashes_from_files(dummy_hashes_file, dummy_wordlist_file, hash_type='sha256')

# Clean up dummy files
# import os
# if os.path.exists(dummy_wordlist_file): os.remove(dummy_wordlist_file)
# if os.path.exists(dummy_hashes_file): os.remove(dummy_hashes_file)
# print("[*] Removed dummy files.")

### 5c. Base64 Encoding/Decoding

**Concept:** Common encoding seen in CTFs, web traffic, etc. Not encryption!

In [ ]:
import base64

def encode_base64(text):
    try:
        encoded_bytes = base64.b64encode(text.encode('utf-8'))
        encoded_string = encoded_bytes.decode('utf-8')
        print(f"[*] Text: '{text}' => Base64: '{encoded_string}'")
        return encoded_string
    except Exception as e:
        print(f"[!] Error encoding: {e}")
        return None

def decode_base64(encoded_string):
    try:
        decoded_bytes = base64.b64decode(encoded_string.encode('utf-8'))
        decoded_string = decoded_bytes.decode('utf-8')
        print(f"[*] Base64: '{encoded_string}' => Text: '{decoded_string}'")
        return decoded_string
    except (base64.binascii.Error, UnicodeDecodeError) as e:
        print(f"[!] Error decoding '{encoded_string}': {e} - Invalid Base64 or cannot decode as UTF-8")
        return None
    except Exception as e:
        print(f"[!] Error decoding: {e}")
        return None

# --- Example Usage ---
original = "This is a secret message for the CTF"
encoded = encode_base64(original)
if encoded:
    decode_base64(encoded)

print("---")
# Example from Week 2 JSON
json_payload = "SFRCe0pTT05fSEVJU1RfRkxBR30=" 
decode_base64(json_payload)

## Week 7: The Hidden Code CTF & Data Encoding/Steganography Basics

**Tasks:** Extract hidden data from a structured file (e.g., CSV).

In [ ]:
import csv

# Assume 'leaked_transactions.csv' exists
# Create a dummy CSV file:
dummy_csv_filename = 'leaked_transactions.csv'
dummy_csv_data = [
    ['Timestamp', 'FromAccount', 'ToAccount', 'Amount', 'Notes'],
    ['2025-04-23 12:00', 'A123', 'B456', '100.00', 'Payment'],
    ['2025-04-23 12:05', 'C789', 'D012', '50.50', 'Transfer'],
    ['2025-04-23 12:10', 'E345', 'F678', '25.00', 'Hidden Data: CTF{CSV_'],
    ['2025-04-23 12:15', 'G901', 'H234', '10.00', 'S3CR3T_FL4G}'], # Flag split across rows/columns?
    ['2025-04-23 12:20', 'I567', 'J890', '75.25', 'Refund']
]
try:
    with open(dummy_csv_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(dummy_csv_data)
    print(f"[*] Created dummy file: {dummy_csv_filename}")
except IOError as e:
    print(f"[!] Error creating dummy file: {e}")

def find_keywords_in_csv(csv_filename, keywords):
    """Reads a CSV file and searches for keywords in any cell."""
    print(f"\n[*] Searching CSV file: {csv_filename} for keywords: {keywords}")
    found_data = []
    try:
        with open(csv_filename, 'r', newline='', encoding='utf-8') as f:
            reader = csv.reader(f)
            header = next(reader, None) # Read the header row
            print(f"[*] Header: {header}")
            
            for row_num, row in enumerate(reader, 1):
                for col_num, cell in enumerate(row):
                    # Check if any keyword is in the cell content
                    if any(keyword.lower() in cell.lower() for keyword in keywords):
                        print(f"[*] Keyword Match at Row {row_num}, Col {col_num}: '{cell}'")
                        found_data.append({'row': row_num, 'col': col_num, 'content': cell, 'full_row': row})
                        # Simple attempt to reconstruct a flag if split
                        if "CTF{" in cell or "}" in cell or "flag" in cell.lower() or "secret" in cell.lower():
                             print(f"    [!] Possible flag fragment found.")
                             
        if not found_data:
            print(f"[!] No cells found containing keywords: {keywords}")
            
        # Add logic here to potentially piece together fragments if needed for the CTF
            
        return found_data
            
    except FileNotFoundError:
        print(f"[!] Error: File '{csv_filename}' not found.")
        return []
    except csv.Error as e:
         print(f"[!] Error reading CSV file '{csv_filename}': {e}")
         return []
    except Exception as e:
        print(f"[!] An unexpected error occurred: {e}")
        return []

# --- Example Usage ---
csv_keywords = ['Hidden', 'Secret', 'Flag', 'CTF{']
find_keywords_in_csv(dummy_csv_filename, csv_keywords)

# Clean up dummy file
# import os
# if os.path.exists(dummy_csv_filename):
#     os.remove(dummy_csv_filename)
#     print(f"[*] Removed dummy file: {dummy_csv_filename}")

## Week 8: Web Traffic Inspector CTF & Network/HAR Analysis

**Task:** Analyze an HTTP Archive (HAR) file (which is JSON format) to extract information.

In [ ]:
import json
import base64

# HAR files are complex JSON. This is a simplified example.
# Create a dummy HAR file:
dummy_har_filename = 'web_traffic.har'
dummy_har_data = {
  "log": {
    "version": "1.2",
    "creator": {"name": "Dummy Browser", "version": "1.0"},
    "entries": [
      {
        "request": {
          "method": "GET",
          "url": "http://example.com/",
          "headers": []
        },
        "response": {
          "status": 200,
          "content": {"mimeType": "text/html", "size": 100, "text": "<html>...</html>"}
        }
      },
      {
        "request": {
          "method": "POST",
          "url": "http://example.com/api/submit",
          "headers": [],
          "postData": {
            "mimeType": "application/x-www-form-urlencoded",
            "text": "user=test&comment=This+is+a+test"
          }
        },
        "response": {
          "status": 200,
          "content": {"mimeType": "application/json", "size": 50, "text": "{\"success\": true}"}
        }
      },
       {
        "request": {
          "method": "GET",
          "url": "http://example.com/admin_notes",
          "headers": [] 
        },
        "response": {
          "status": 200,
          "content": {
               "mimeType": "text/plain", 
               "size": 50, 
                # Sometimes sensitive data might be base64 encoded in content
               "text": "Q1RGe0hBUl9BTkFMWVNJU19GT एफBhZ30=", 
               "encoding": "base64" 
               # Flag: CTF{HAR_ANALYSIS_FLAG}
           }
        }
      }
    ]
  }
}
try:
    with open(dummy_har_filename, 'w') as f:
        json.dump(dummy_har_data, f, indent=2)
    print(f"[*] Created dummy file: {dummy_har_filename}")
except IOError as e:
    print(f"[!] Error creating dummy file: {e}")

def analyze_har_file(har_filename, search_terms):
    """Basic analysis of a HAR file for keywords in URLs, request/response bodies."""
    print(f"\n[*] Analyzing HAR file: {har_filename} for terms: {search_terms}")
    try:
        with open(har_filename, 'r', encoding='utf-8') as f:
            har_data = json.load(f)
            
        if 'log' not in har_data or 'entries' not in har_data['log']:
            print("[!] Invalid HAR structure: Missing 'log' or 'entries'.")
            return
        
        for i, entry in enumerate(har_data['log']['entries']):
            request = entry.get('request', {})
            response = entry.get('response', {})
            url = request.get('url', '')
            method = request.get('method', '')
            
            # Check URL
            if any(term.lower() in url.lower() for term in search_terms):
                print(f"[*] Match in URL (Entry {i}, {method}): {url}")
            
            # Check Request Body (postData)
            post_data = request.get('postData', {})
            request_body = post_data.get('text', '')
            if request_body and any(term.lower() in request_body.lower() for term in search_terms):
                 print(f"[*] Match in Request Body (Entry {i}, {method} {url}): {request_body[:200]}...")

            # Check Response Body
            content = response.get('content', {})
            response_body = content.get('text', '')
            encoding = content.get('encoding')
            
            decoded_response_body = None
            if response_body:
                 # Handle potential Base64 encoding mentioned in HAR
                 if encoding == 'base64':
                     try:
                         decoded_response_body = base64.b64decode(response_body).decode('utf-8', errors='ignore')
                     except Exception as e:
                         print(f"[!] Error Base64 decoding response body (Entry {i}): {e}")
                         decoded_response_body = response_body # Fallback to original if decode fails
                 else:
                    decoded_response_body = response_body
            
                 # Search in the (potentially decoded) response body
                 if decoded_response_body and any(term.lower() in decoded_response_body.lower() for term in search_terms):
                     print(f"[*] Match in Response Body (Entry {i}, {method} {url}): {decoded_response_body[:200]}...")
                     # Check if it looks like a flag
                     if "CTF{" in decoded_response_body:
                          start_index = decoded_response_body.find("CTF{")
                          end_index = decoded_response_body.find('}', start_index)
                          if end_index != -1:
                              flag = decoded_response_body[start_index:end_index+1].strip()
                              print(f"    [!] Potential Flag Extracted: {flag}")
                             
    except FileNotFoundError:
        print(f"[!] Error: File '{har_filename}' not found.")
    except json.JSONDecodeError as e:
        print(f"[!] Error decoding JSON from '{har_filename}': {e}")
    except Exception as e:
        print(f"[!] An unexpected error occurred: {e}")

# --- Example Usage ---
har_search_terms = ['admin', 'password', 'secret', 'token', 'flag', 'CTF{', 'key']
analyze_har_file(dummy_har_filename, har_search_terms)

# Clean up dummy file
# import os
# if os.path.exists(dummy_har_filename):
#     os.remove(dummy_har_filename)
#     print(f"[*] Removed dummy file: {dummy_har_filename}")

## Weeks 9-10: Milestone 2 (Trace the Code) & Debugging

**Task:** Simplify a 'messy' script to reveal a hidden flag. Often involves understanding obfuscation or basic debugging.

In [ ]:
import base64

# Example of a 'messy' script with simple obfuscation
# Assume this is content of 'Existing Script.py'
messy_script_content = """
import base64

# Some complex calculations or irrelevant code...
x = 10
y = 20
for i in range(5):
    x += i * y
    # print(f'Intermediate: {x}') # Debugging print often commented out

# Obfuscated flag using Base64
part1 = 'Q1RGe1RSQUNFX1'
part2 = 'RIRfQ09ERV9TS'
part3 = 'UxWQ0VEX0ZMQ'
part4 = 'Ud9'

def reveal():
    # Concatenate parts
    full_encoded = part1 + part2 + part3 + part4
    # print(f'Full encoded: {full_encoded}') # Another potential debug point
    try:
        decoded_bytes = base64.b64decode(full_encoded)
        flag = decoded_bytes.decode('utf-8')
        # The script might hide the output or make it hard to find
        # print(f'The secret is: {flag}') 
        # Instead, maybe it writes to a weird file or just returns it?
        return flag 
    except Exception:
        return 'Decoding Failed'

# The script might call reveal() but not print the result directly
hidden_value = reveal()
# Maybe it does something else with hidden_value later...
# if 'CTF' in hidden_value:
#     print('Success path')
# else:
#     print('Failure path')
"""

dummy_script_filename = 'Existing Script.py'
try:
    with open(dummy_script_filename, 'w') as f:
        f.write(messy_script_content)
    print(f"[*] Created dummy file: {dummy_script_filename}")
except IOError as e:
    print(f"[!] Error creating dummy file: {e}")


# --- Simulating the Analysis Process --- 
print(f"\n[*] Analyzing the 'messy' script ('{dummy_script_filename}')")

# 1. Identify suspicious parts: Base64 strings, concatenation, decoding function
p1 = 'Q1RGe1RSQUNFX1'
p2 = 'RIRfQ09ERV9TS'
p3 = 'UxWQ0VEX0ZMQ'
p4 = 'Ud9'
combined_encoded = p1 + p2 + p3 + p4
print(f"[*] Identified combined Base64 string: {combined_encoded}")

# 2. Manually decode the suspicious part
try:
    decoded_bytes = base64.b64decode(combined_encoded)
    decoded_flag = decoded_bytes.decode('utf-8')
    print(f"[*] Manually Decoded Flag: {decoded_flag}")
    print("    [!] This looks like the flag!")
except Exception as e:
    print(f"[!] Manual decoding failed: {e}")

# 3. Alternative: Import and run the function if possible (or use debugger)
print("\n[*] Trying to import and run the 'reveal' function from the script...")
try:
    # Be cautious importing unknown scripts!
    import importlib
    # Need to ensure the module name matches the filename without '.py'
    module_name = dummy_script_filename.replace('.py', '')
    # Invalidate caches in case the file changed
    importlib.invalidate_caches()
    # Import the module
    messy_module = importlib.import_module(module_name)
    
    # Call the function
    if hasattr(messy_module, 'reveal'):
        result = messy_module.reveal()
        print(f"[*] Result from calling reveal(): {result}")
    else:
        print("[!] Could not find 'reveal' function in the module.")
        
except ModuleNotFoundError:
     print(f"[!] Could not import module '{module_name}'. Make sure it's in Python's path.")
except Exception as e:
     print(f"[!] Error importing/running script function: {e}")

# Clean up dummy file
# import os
# import sys
# if os.path.exists(dummy_script_filename): os.remove(dummy_script_filename)
# # Remove .pyc file if it exists
# pyc_file = dummy_script_filename + 'c' # Basic check, might be in __pycache__
# if os.path.exists(pyc_file): os.remove(pyc_file)
# # Remove from imported modules if necessary
# if module_name in sys.modules: del sys.modules[module_name] 
# print("[*] Removed dummy script and related files/modules.")

## Conclusion

This notebook provides Python snippets directly aligned with the concepts and CTF challenges outlined in your **MO-IT143 Ethical Hacking** syllabus. Practice these techniques, focusing on understanding *how* they work and how they apply to identifying and analyzing potential security issues.

**Key Takeaways from Syllabus:**
*   Python is crucial for automating file parsing (text, JSON, CSV, logs, HAR).
*   Understanding web requests (`requests`) is essential for web interaction and vulnerability testing concepts (like SQLi).
*   Basic cryptography (hashing, encoding like Base64) is common in CTFs.
*   Debugging and analyzing existing code is a valuable skill (Milestone 2).

**Always remember ethical considerations and only test on systems you have explicit permission for.** Good luck with the course and the CTFs!